In [2]:
from environments import DeepSea
import numpy as np
from tqdm.notebook import tqdm

In [6]:
H = 10
env = DeepSea(H)
cost = 0.0
tuples = []
for k in tqdm(range(int(5*10*2**H))):
    s,done = env.reset()
    while not done:
        a = np.random.binomial(1,0.1)
        c,s_,done = env.step(s,a)
        cost += c
        tuples.append([s,a,c,s_])
        s = s_

  0%|          | 0/51200 [00:00<?, ?it/s]

In [7]:
class Fitted_Q_Iteration(object):
    def __init__(self,data,loss,iters,nState,nAction,gamma):
        self.data = data.copy()
        self.loss = loss
        self.iters = iters
        self.nState = nState
        self.nAction = nAction
        self.gamma = gamma
        self.d = nState*nAction
        self.n = len(data)
        self.phi = np.identity(nState*nAction)
        self.f = np.zeros((nState,nAction))
        self.f_ = np.zeros((nState,nAction)) + 0.1
        self.Vf = np.zeros(nState)
        self.get_feature_idx()
        self.getA()
        self.Q_ensemble = np.random.uniform(size=(100,nState,nAction))
        
    
    def get_feature_idx(self):
        self.tuple_to_idx = {}
        self.idx_to_tuple = {}
        k = 0
        for s in range(self.nState):
            for a in range(self.nAction):
                self.tuple_to_idx[s,a] = k
                self.tuple_to_idx[k] = [s,a]
                k += 1
    
    def getA(self):
        data = self.data.copy()
        self.A = np.zeros((self.d,self.d))
        for i in range(self.n):
            s,a = data[i][0],data[i][1]
            idx = self.tuple_to_idx[s,a]
            self.A[idx,idx] += 1
        
    
    def get_targets(self):
        self.tar = np.zeros( self.n )
        data = self.data
        for i in range(self.n):
            c, s_ = data[i][2], data[i][3]
            if s_ != None:
                self.tar[i] = c + self.gamma * min(self.f_[s_,:])
            else:
                self.tar[i] = c
                
    def minimize_f_sq(self):
        data = self.data.copy()
        b = np.zeros(self.d)
        for i in range(self.n):
            s,a = data[i][0],data[i][1]
            idx = self.tuple_to_idx[s,a]
            b[idx] += self.tar[i]
        f = np.linalg.solve(self.A,b)
        self.f = np.reshape(f,(self.nState,self.nAction))
    
    
    def minimize_f_log(self):
        pass
            
    
    def update_Q_log(self):
        self.get_targets()
        self.minimize_f_log()
        self.f_ = self.f
    
    def update_Q_sq(self):
        self.get_targets()
        self.minimize_f_sq()
        self.f_ = self.f
    
    def update_Q(self):
        if self.loss == 'log':
            self.update_Q_log()
        else:
            self.update_Q_sq()
    
    def run(self):
        for k in tqdm(range(self.iters)):
            self.update_Q()
        

In [8]:
nState = len(env.state_to_tuple)
agent = Fitted_Q_Iteration(tuples,'square',20,nState,2,0.99)
agent.run()
print(agent.f)

  0%|          | 0/20 [00:00<?, ?it/s]

LinAlgError: Singular matrix

In [1]:
tuples

NameError: name 'tuples' is not defined